# Assignment 3 : Computation of length scales and spectrum in space

We have fluctuations of velocity in time, but not in space. In order to get fluctuations of velocity in space (for a given time $t_0$) we should have an array of anemometers very close to each other (microns, probably), which is quite difficult (it is much more easier for CFD results). 

Fortunately, there is the [Taylor hypothesis](https://www-sciencedirect-com.recursos.biblioteca.upc.edu/science/article/pii/B9780080443508500014#p15) (see also Pope's book reference at the beginning of the chapter, p. 223), which says that if turbulence can be considered (average) steady and fluctuations are much smaller than average velocity, time can be replaced by $x/\overline{u}$. The we can estimate the lenght scales of turbulence by replacing $t$ by $x$ and frequency $f$ by wavenumber $k$ (do not confuse with turbulent kinetic energy).

In this assignment it is proposed to to make all the previous analysis and find, in space:

- Kolmogorov's scale
- Autocorrelation function
- Integral scale
- Power spectrum (in $k$) and check the Kolmogorov's power law.

Do not do that for $y = 0 \,\text{mm}$. Choose another $y$ !

---------------------------------
# A3: GROUP 1 SOLUTION (Pol Padilla, Ferran de Miguel, Alejandro Sanchez)
---------------------------------

In [111]:
import pandas as pd
import numpy as np

In [112]:
newTurbulenceData = pd.read_hdf("TurbVel.hdf")
newTurbulenceData

,y = -40mm,y = -35mm,y = -30mm,y = -25mm,y = -20mm,y = -15mm,y = -10mm,y = -5mm,y = 0mm,y = 5mm,y = 10mm,y = 15mm,y = 20mm,y = 25mm,y = 30mm,y = 35mm,y = 40mm
Time,,,,,,,,,,,,,,,,,
0.000,20.916005,21.013892,19.890369,20.564816,19.247617,20.242762,20.171834,19.326923,17.711748,17.954515,19.112384,19.247617,18.955669,19.624038,18.989162,20.337691,17.901476
0.001,20.757920,21.124463,19.878740,20.528820,19.258909,20.278330,20.124676,19.417902,16.314876,17.534186,19.213710,19.292888,18.922264,19.704760,18.833365,20.313920,17.743229
0.002,20.661140,21.087559,20.101135,20.492846,19.213710,20.337691,20.136438,19.360977,13.902128,18.590930,19.202443,19.349631,18.933382,19.716296,18.601871,20.254624,17.922668
0.003,20.794288,21.099841,20.207287,20.421109,19.315560,20.219094,20.124676,19.349631,12.840679,18.340470,19.281544,19.372364,18.955669,19.785785,18.503472,20.278330,17.471901
0.004,20.952667,21.099841,20.349604,20.480892,19.292888,20.313920,20.148242,19.326923,15.086783,18.243283,19.247617,19.349631,18.944540,19.785785,17.838033,20.195451,18.135857
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
0.995,21.087559,21.248015,19.349631,20.007225,19.785785,19.832212,19.612552,18.844470,17.010890,18.877784,17.234927,19.937022,20.373413,18.800155,17.451182,18.922264,19.808986
0.996,21.013892,21.334855,19.474945,19.972095,19.774176,19.855463,19.601036,18.844470,16.960352,18.082340,17.513403,20.183621,20.148242,18.800155,17.440815,19.532177,20.207287
0.997,21.050677,21.310005,19.440708,20.054129,19.704760,19.867080,19.578058,18.888878,16.373577,18.623839,17.680285,19.960385,20.148242,18.329634,17.461523,19.797365,20.160017


In [113]:
statData = newTurbulenceData.describe()
statData

,y = -40mm,y = -35mm,y = -30mm,y = -25mm,y = -20mm,y = -15mm,y = -10mm,y = -5mm,y = 0mm,y = 5mm,y = 10mm,y = 15mm,y = 20mm,y = 25mm,y = 30mm,y = 35mm,y = 40mm
count,1000.000000,1000.000000,1000.000000,1000.000000,1000.000000,1000.000000,1000.000000,1000.000000,1000.000000,1000.000000,1000.000000,1000.000000,1000.000000,1000.000000,1000.000000,1000.000000,1000.000000
mean,21.104367,21.044473,20.757320,20.316435,20.247308,20.270573,19.733937,19.493431,17.798975,17.059820,19.138614,19.377307,19.330135,19.228752,19.026175,19.121565,18.570242
std,0.187226,0.228523,0.762061,0.776583,0.536762,0.262442,0.251107,0.491538,1.124547,0.988907,0.305786,0.354726,0.462051,0.478166,0.694788,0.662401,0.860208
min,20.468907,20.207287,18.634798,18.866662,19.123603,19.704760,19.168608,18.297226,12.824605,13.765816,17.234927,17.764286,17.245155,17.265657,16.373577,16.511184,16.101291
25%,20.989390,20.989390,20.322845,19.751012,19.797365,20.065853,19.566596,19.075895,17.327294,16.491465,18.977981,19.213710,19.109572,18.989162,18.678760,18.722815,17.983749
50%,21.099841,21.112168,20.733695,20.225018,20.272393,20.219094,19.704760,19.532177,18.071674,17.173579,19.168608,19.440708,19.338256,19.258909,19.045057,19.258909,18.689782
75%,21.173826,21.186155,21.062977,20.624988,20.676252,20.492846,19.913683,19.937022,18.569031,17.774791,19.315560,19.601036,19.658606,19.520739,19.477807,19.601036,19.227838
max,21.647802,21.447038,23.727658,23.157105,21.774209,20.842895,20.361487,20.661140,19.843817,19.727874,20.077620,20.230943,20.721574,20.444995,20.830752,20.337691,20.492846


In [114]:

timeYdata = newTurbulenceData[['y = 5mm']]
timeYdata

,y = 5mm
Time,
0.000,17.954515
0.001,17.534186
0.002,18.590930
0.003,18.340470
0.004,18.243283
...,...
0.995,18.877784
0.996,18.082340
0.997,18.623839


In [115]:
Umean=statData.loc['mean']['y = 5mm']
Umean

17.059820449724764

In [116]:
spaceXdata = timeYdata

list11 = list(map(float,spaceXdata.index.values))
print(list11)

list2 = []
for i in list11:
    list2.append(i * Umean)

spaceXdata = spaceXdata.rename(index=dict(zip(list11,list2)))
spaceXdata.index.name="Space"
spaceXdata



[0.0, 0.001, 0.002, 0.003, 0.004, 0.005, 0.006, 0.007, 0.008, 0.009, 0.01, 0.011, 0.012, 0.013, 0.014, 0.015, 0.016, 0.017, 0.018, 0.019, 0.02, 0.021, 0.022, 0.023, 0.024, 0.025, 0.026, 0.027, 0.028, 0.029, 0.03, 0.031, 0.032, 0.033, 0.034, 0.035, 0.036, 0.037, 0.038, 0.039, 0.04, 0.041, 0.042, 0.043, 0.044, 0.045, 0.046, 0.047, 0.048, 0.049, 0.05, 0.051, 0.052, 0.053, 0.054, 0.055, 0.056, 0.057, 0.058, 0.059, 0.06, 0.061, 0.062, 0.063, 0.064, 0.065, 0.066, 0.067, 0.068, 0.069, 0.07, 0.071, 0.072, 0.073, 0.074, 0.075, 0.076, 0.077, 0.078, 0.079, 0.08, 0.081, 0.082, 0.083, 0.084, 0.085, 0.086, 0.087, 0.088, 0.089, 0.09, 0.091, 0.092, 0.093, 0.094, 0.095, 0.096, 0.097, 0.098, 0.099, 0.1, 0.101, 0.102, 0.103, 0.104, 0.105, 0.106, 0.107, 0.108, 0.109, 0.11, 0.111, 0.112, 0.113, 0.114, 0.115, 0.116, 0.117, 0.118, 0.119, 0.12, 0.121, 0.122, 0.123, 0.124, 0.125, 0.126, 0.127, 0.128, 0.129, 0.13, 0.131, 0.132, 0.133, 0.134, 0.135, 0.136, 0.137, 0.138, 0.139, 0.14, 0.141, 0.142, 0.143, 0.144, 0

,y = 5mm
Space,
0.000000,17.954515
0.017060,17.534186
0.034120,18.590930
0.051179,18.340470
0.068239,18.243283
...,...
16.974521,18.877784
16.991581,18.082340
17.008641,18.623839
